# 2. Data preparation

---

### Setup

In [1]:
import pandas as pd

original: pd.DataFrame = pd.read_csv('qsar_oral_toxicity_after_profiling.csv', sep=';')

## 2.0. Data preparation

---

### Missing Values Imputation

---

### Scaling

---

Since all variables are binary (0 or 1) they are already scaled. Therefore, no scaling is needed.

In [2]:
original.to_csv('qsar_oral_toxicity_after_preparation.csv', sep=';', index=False)